In [1]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, multilabel_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from keras_preprocessing.sequence import pad_sequences
from varname import nameof
from numpy import savetxt
import seaborn as sns
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import wget

from numpy import asarray
from numpy import savetxt

import geopandas as geop

import glob
import os

from datetime import date

import pymysql

In [22]:
path = r"D:\Materi Kuliah\SKRIPSI"
os.chdir(path)

current_directory = os.getcwd()
print("Direktori kerja saat ini:", current_directory)

Direktori kerja saat ini: D:\Materi Kuliah\SKRIPSI


In [5]:
parameter_geo=pd.read_csv("Data/Data Spatial/parameter_geo.csv")
# Menghapus spasi dan menggabungkan kata-kata pada kolom "Propinsi"
parameter_geo['Propinsi'] = parameter_geo['Propinsi'].replace(' ', '', regex=True)
for i in range(len(parameter_geo)):
    today = "2023-06-16"#date.today()
    yesterday = "2023-06-17"#today - timedelta(days = 1)
    tokoh="Anies Baswedan"
    lokasi=str(parameter_geo.Propinsi[i])
    radius=str(parameter_geo.radius[i])+"km"
       
    twitter_search = f"{tokoh} near:{lokasi} within:{radius} lang:id until:{yesterday} since:{today}"

    # Tentukan nama file dengan format "<kueri pencarian>_<tanggal saat ini>.json"
    filename = "Data\Data JSON\Anies.json"
    USING_TOP_SEARCH = True

    snscrape_params = '--jsonl'
    twitter_search_params = ''

    if USING_TOP_SEARCH:
        twitter_search_params += "--top"

    os.system(f"cmd /c snscrape {snscrape_params} twitter-search {twitter_search_params} \"{twitter_search}\" > {filename}")
    tweets_df = pd.read_json(filename, lines=True)
    if len(tweets_df) != 0:    
        tweets_df_rename=tweets_df.rename(columns={"date": "date", "username": "username", "rawContent": "tweet"})
        tweets_df_fix=tweets_df_rename[["date","username", "tweet"]]
        tweets_df_fix["location"] = lokasi
        tweets_df_fix.to_csv(f"Data/Hasil Crawling/{lokasi}_ANIES.csv", index=False)

In [14]:
df_anies= pd.concat(map(pd.read_csv, glob.glob(os.path.join('Data/Hasil Crawling/', "*_ANIES.csv"))))
df_anies= df_anies.reset_index(drop=True)
df_anies = df_anies[["date", "username", "tweet", "location"]]
df_anies.to_csv("Data/Hasil Crawling/data_anies.csv", index=False)

In [18]:
anies=pd.read_csv('Data/Hasil Crawling/data_anies.csv')
anies.tail()

,date,username,tweet,location
15446,2023-06-16 13:09:53+00:00,DoankWarto,Di sini kelihatan bacapres yg dikehendaki raky...,SUMATERAUTARA
15447,2023-06-16 06:44:29+00:00,KimAnkiim,“Jawa Timur ingin perubahan bersama Anies Basw...,SUMATERAUTARA
15448,2023-06-16 23:33:03+00:00,hahahabud,@Semot_x_Abang @juli_rahardjo @tvOneNews @anie...,SUMATERAUTARA
15449,2023-06-16 02:39:51+00:00,deathnoteee_,Anies terburu-buru potong pita.\n\n#tolakanies...,SUMATERAUTARA
15450,2023-06-16 11:32:44+00:00,buduuum,@aniesbaswedan Alhamdulillah\n---\nhttps://t.c...,SUMATERAUTARA


In [20]:
# len(anies[anies["location"]=="SUMATERAUTARA"])

623

In [21]:
def replaceSlang(word):
    if word in list(dataslang[0]):
        indexslang = list(dataslang[0]).index(word)
        return dataslang[1][indexslang]
    else:
        return word

def removeStopWords(line, stopwords):
    words = []
    for word in line:  
        word=str(word)
        word = word.strip()
        if word not in stopwords and word != "" and word != "&":
            words.append(word)
    return words

def stemmer(line):
    temp = list()
    for word in line:
        if(word not in white_list):
            word = ind_stemmer.stem(word)
        if(len(word)>3):
            temp.append(word)
        return temp

def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength

def insert_to_list_rekomen_db(rp2):
    # Connect to the database
    connection = pymysql.connect(host='127.0.0.1',
                                port=3306,
                                user='root',
                                # password='f#Ur8J3N',
                                database='klasifikasi_sentimen')
    # create cursor
    cursor=connection.cursor()
        
    cols = "`,`".join([str(i) for i in rp2.columns.tolist()])
    for i,row in rp2.iterrows():
        sql = "INSERT INTO `hasil_scraping` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()



# inisiasi variabel awal
start = 0
end = 1000

# loop untuk membagi data menjadi bagian-bagian dengan jumlah baris 1000
while end <= len(anies):
    df_anies = anies.iloc[start:end, 0:4]
    #menyimpan tweet. (tipe data series pandas)
    data_content = df_anies['tweet']
    # casefolding
    data_casefolding = data_content.str.lower()
    #data_casefolding.head()
    #filtering

    #url
    filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
    #cont
    filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
    #punctuatuion
    filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
    #  hapus #tagger
    filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
    #numeric
    filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

    # # filtering RT , @ dan #
    # fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
    # clean = [fungsi_clen_rt for tweet in filtering_numeric]

    data_filtering = pd.Series(filtering_numeric)

    # #tokenize
    tknzr = TweetTokenizer()
    data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

    #slang word
    path_dataslang = open("Data/Data Tambahan/kamus kata baku-clear (1).csv")
    dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

    data_formal = []
    for data in data_tokenize:
        data_clean = [replaceSlang(word) for word in data]
        data_formal.append(data_clean)
    len_data_formal = len(data_formal)
    # print(data_formal)
    # len_data_formal
    
    nltk.download('stopwords')
    default_stop_words = nltk.corpus.stopwords.words('indonesian')
    stopwords = set(default_stop_words)

    
    data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]
    white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

    factory = StemmerFactory()
    ind_stemmer = factory.create_stemmer()


    reviews = [stemmer (line) for line in data_notstopword]
    
    #Pembuatan Kamus kata
    t  = Tokenizer()
    fit_text = reviews
    t.fit_on_texts(fit_text)

    #Pembuatan Id masing-masing kata
    sequences = t.texts_to_sequences(reviews)

    #hapus duplikat kata yang muncul
    list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

    #mencari max length sequence

        
    # Driver Code 
    max_seq, max_length_seq = FindMaxLength(list_set_sequence)
    jumlah_index = len(t.word_index) +1

    print('jumlah index : ',jumlah_index,'\n')
    # print('word_index : ',t.word_index,'\n')
    # print('index kalimat asli     : ', sequences,'\n')
    # print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
    # print('panjang max kalimat : ', max_length_seq,'kata','\n')
    # print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

    count_word = [len(i) for i in list_set_sequence]
    # print('list panjang kalimat : ', count_word)
    max_len_word = max(count_word)
    # print(max_len_word)
    
    padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 50, padding='pre')
    
    nama_model='gabungan'
    model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
    result=[]
    for test in padding:
        pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
        result.append(pred[0])
    df_anies['Tokoh'] = 'Anies Baswedan'
    df_anies['Sentiment'] = result
    df_anies.loc[df_anies['Sentiment'] == 2, 'Sentiment'] = -1
                
    insert_to_list_rekomen_db(df_anies)
    start += 1000
    end += 1000

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  230 

1/1 [==============================] - 0s 19ms/step


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  158 

1/1 [==============================] - 0s 22ms/step


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


...

jumlah index :  51 

1/1 [==============================] - 0s 19ms/step


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  64 

1/1 [==============================] - 0s 19ms/step


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  51 

1/1 [==============================] - 0s 18ms/step


In [6]:
#menyimpan tweet. (tipe data series pandas)
data_content = df_anies['tweet']
# casefolding
data_casefolding = data_content.str.lower()
#data_casefolding.head()
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)

# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

#slang word
path_dataslang = open("dataset/kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

KeyboardInterrupt: 

In [ ]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
# print(reviews)

In [ ]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
# print('word_index : ',t.word_index,'\n')
# print('index kalimat asli     : ', sequences,'\n')
# print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
# print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
# print(max_len_word)

jumlah index :  2229 

panjang max kalimat :  35 kata 



In [ ]:
padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 50, padding='pre')

In [ ]:
nama_model='gabungan'
model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
result=[]
for test in padding:
  pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
  result.append(pred[0])

1/1 [==============================] - 0s 46ms/step


In [ ]:
df_anies['Tokoh'] = 'Anies Baswedan'
df_anies['Sentiment'] = result
df_anies.loc[df_anies['Sentiment'] == 2, 'Sentiment'] = -1
df_anies.head()

,date,username,tweet,location,Tokoh,Sentiment
0,2023-03-24 05:59:26,NyakBanda,@NahiMungkar5 @aniesbaswedan Emangnya ada yg P...,ACEH,Anies Baswedan,0
1,2023-03-24 05:42:51,GheaJhanaLie8,Media memainkan peran yg sangat vital untuk me...,ACEH,Anies Baswedan,-1
2,2023-03-24 04:09:31,BSipoel,@AyaniMel Disaat yang lain menebar isu perpeca...,ACEH,Anies Baswedan,-1
3,2023-03-24 04:08:16,BSipoel,@VeelaaRhie Anies Baswedan mengayomi seluruh l...,ACEH,Anies Baswedan,0
4,2023-03-24 03:51:33,BSipoel,@VeelaaRhie Mantap dukung Anies Baswedan for R...,ACEH,Anies Baswedan,-1


In [ ]:
df_anies['Sentiment'].unique()

array([ 0, -1,  1], dtype=int64)

In [14]:
def insert_to_list_rekomen_db(rp2):
    # Connect to the database
    connection = pymysql.connect(host='127.0.0.1',
                                 port=3306,
                                 user='root',
                                 # password='f#Ur8J3N',
                                 database='klasifikasi_sentimen')
    # create cursor
    cursor=connection.cursor()
    
    cols = "`,`".join([str(i) for i in rp2.columns.tolist()])
    for i,row in rp2.iterrows():
        sql = "INSERT INTO `hasil_scraping` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

In [15]:
insert_to_list_rekomen_db(rp2=df_anies)